### Task 2: AdaBoost (Adaptive Boosting)
#### **Objective**
In this task, you will implement the **AdaBoost** algorithm for binary classification. AdaBoost is an ensemble method that combines multiple "weak learners" (typically simple decision trees) to create a strong classifier.

#### **Overview**
The AdaBoost algorithm works by iteratively training weak learners on the dataset. In each iteration, it adjusts the weights of the training samples so that the next learner focuses more on the samples that were misclassified by the previous ones.

The algorithm proceeds as follows (for $T$ rounds):
1. **Initialize weights**: Assign equal weights $w_i = 1/n$ to all $n$ training samples.
2. **Iterate $t=1$ to $T$**:
   - **(a) Fit a weak learner**: Train a base classifier $h_t$ using the weighted samples.
   - **(b) Compute weighted error**: Calculate the error $\varepsilon_t$ as the sum of weights of misclassified samples.
   - **(c) Compute learner weight**: Calculate $\alpha_t = \frac{1}{2} \ln\left(\frac{1-\varepsilon_t}{\varepsilon_t}\right)$.
   - **(d) Update sample weights**: Increase weights for misclassified samples: $w_i \leftarrow w_i \cdot \exp(-\alpha_t y_i h_t(x_i))$.
   - **(e) Normalize weights**: Scale $w_i$ so that $\sum w_i = 1$.
3. **Final Prediction**: The final model aggregates predictions: $F_T(x) = \sum_{t=1}^T \alpha_t h_t(x)$, and the predicted class is $\text{sign}(F_T(x))$.

---
#### **SubTasks**

##### **SubTask 1: Compute Weighted Error**
- **Function to complete**: `_compute_error(self, y, y_pred, w)`
- **Purpose**: Calculate the total weight of misclassified samples.
- **Formula**: $\varepsilon_t = \sum_{i=1}^n w_i \cdot \mathbb{1}(h_t(x_i) \neq y_i)$

##### **SubTask 2: Compute Learner Weight**
- **Function to complete**: `_compute_alpha(self, error)`
- **Purpose**: Determine the importance ($\alpha_t$) of the current weak learner based on its error.
- **Formula**: $\alpha_t = \frac{1}{2} \ln\left(\frac{1-\varepsilon_t}{\varepsilon_t}\right)$

##### **SubTask 3: Update Sample Weights**
- **Function to complete**: `_update_weights(self, w, alpha, y, y_pred)`
- **Purpose**: Update weights to penalize misclassifications and normalize them.
- **Formula**: $w_i \leftarrow w_i \cdot \exp(-\alpha_t y_i h_t(x_i))$, followed by normalization.

##### **SubTask 4: Train AdaBoost**
- **Function to complete**: `fit(self, X, y)`
- **Purpose**: Implement the main loop of the AdaBoost algorithm.
- **Steps**:
  1. Initialize weights $w$.
  2. Loop $T$ times:
     - Train weak learner.
     - Compute error and alpha.
     - Update weights.
     - Store learner and alpha.


In [1]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier

class AdaBoost:
    """
    AdaBoost (Adaptive Boosting) classifier for binary classification.
    """
    
    def __init__(self, n_estimators=50, seed=None):
        self.n_estimators = n_estimators
        self.seed = seed
        self.alphas = []
        self.models = []
        
    def _compute_error(self, y, y_pred, w):
        """
        Calculates the weighted training error.
        """
        # TODO: Implement this function
        # Calculate the weighted sum of incorrect predictions
        error = np.sum(w *(y_pred != y))
        return error
    
    def _compute_alpha(self, error):
        """
        Computes the weight (alpha) of the current weak learner.
        """
        epsilon = 1e-10  # Tiny constant to avoid division by zero
        # TODO: Implement this function
        # Calculate alpha based on the error
        eps = max(epsilon, error)
        alpha = 0.5 * np.log((1 - eps) / (eps))
        return alpha
    
    def _update_weights(self, w, alpha, y, y_pred):
        """
        Updates and normalizes sample weights.
        """
        # TODO: Implement this function
        # Step 1: Update weights (increase weight for misclassified samples)
        # Hint: y * y_pred is 1 if correct, -1 if incorrect
        w = w * np.exp(-alpha * y * y_pred)
        
        # Step 2: Normalize weights so they sum to 1
        w = w / np.sum(w)
        
        return w
        
    def fit(self, X, y):
        """
        Trains the AdaBoost model.
        """
        n_samples, n_features = X.shape
        
        # Clear previous models
        self.models = []
        self.alphas = []
        
        # IMPORTANT: Convert labels to {-1, 1} if they are {0, 1}
        if set(np.unique(y)) == {0, 1}:
            y = np.where(y == 0, -1, 1)
            
        # Initialize weights uniformly: w_i = 1/n
        w = np.full(n_samples, 1.0 / n_samples)
        
        if self.seed is not None:
            np.random.seed(self.seed)
            
        for t in range(self.n_estimators):
            # a) Fit a weak learner on weighted data
            stump = DecisionTreeClassifier(max_depth=1, random_state=self.seed)
            stump.fit(X, y, sample_weight=w)
            
            # Predict on training data to compute error
            y_pred = stump.predict(X)
            
            # TODO: Implement the boosting steps
            # b) call _compute_error
            error = self._compute_error(y, y_pred, w)
            
            # c) call _compute_alpha
            alpha = self._compute_alpha(error)
            
            # d) call _update_weights
            w = self._update_weights(w, alpha, y, y_pred)
            
            # Store the trained model and its weight
            self.models.append(stump)
            self.alphas.append(alpha)
            
    def predict(self, X):
        """
        Predicts class labels for X.
        """
        # Collect predictions from all weak learners
        weak_preds = np.array([stump.predict(X) for stump in self.models])
        
        # Weighted sum of predictions
        weighted_sum = np.dot(self.alphas, weak_preds)
        
        # Return sign of the sum
        final_preds = np.sign(weighted_sum)
        
        # Map 0s to 1s (rare edge case where sum is exactly 0)
        final_preds[final_preds == 0] = 1
        
        return final_preds

### Verification
Let's test your implementation on the Breast Cancer dataset.

In [2]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load data
data = load_breast_cancer()
X = data.data
y = data.target

# Convert y from {0, 1} to {-1, 1}
y = np.where(y == 0, -1, 1)

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train
clf = AdaBoost(n_estimators=50, seed=42)
clf.fit(X_train, y_train)

# Predict
y_pred = clf.predict(X_test)

# Evaluate
print(f"Test Accuracy: {accuracy_score(y_test, y_pred):.4f}")

Test Accuracy: 0.9649
